In [8]:
from __future__ import division, print_function
from builtins import input

import numpy as np
import sporco as sp

from sporco.dictlrn import prlcnscdl
from sporco import util
# from sporco import signal
# from sporco import array
from sporco import plot
import matplotlib.pyplot as plt
plot.config_notebook_plotting()
import pandas as pd
import pickle

In [9]:
TRAIN_DIR_PREFIX = "/home/harry/Projects/ece657_proj/train_data_"
TEST_DIR_PREFIX = "/home/harry/Projects/ece657_proj/test_data_"
DATA_PATH = [
    "province",
    "area",
    "letter"
]

npd = 5
fltlmbd = 100

In [10]:
def generate_dict(path):
    data = pd.read_csv(path)
    data = data.values
    image=[]
    for i in range(data.shape[0]):
        i=data[i][0:-1].astype("float32")/255
        i=i.reshape(20,20)
        image.append(i)

    S = np.dstack(image)
    
    sl, sh = util.tikhonov_filter(S, fltlmbd, npd)

    np.random.seed(12345)
    D0 = np.random.randn(4, 4, 25)

    lmbda = 0.2
    opt = prlcnscdl.ConvBPDNDictLearn_Consensus.Options({
            'Verbose': False, 'MaxMainIter': 250,
            'CBPDN': {'rho': 50.0*lmbda + 0.5},
            'CCMOD': {'rho': 10.0, 'ZeroMean': True}})
    d = prlcnscdl.ConvBPDNDictLearn_Consensus(D0, sh, lmbda, opt)
    D1 = d.solve()
    print("ConvBPDNDictLearn solve time: %.2fs" % d.timer.elapsed('solve'))
    return D1.squeeze()

In [11]:
from sporco.admm import cbpdn
lmbda = 5e-2
L = 1e2
opt = cbpdn.ConvBPDN.Options({'Verbose': False, 'MaxMainIter': 200,
                              'RelStopTol': 5e-3, 'AuxVarObj': False})


In [12]:
from tqdm.notebook import tqdm
from sporco.admm import cbpdn
def parse_repre(img, D):
    sl, sh = util.tikhonov_filter(img, fltlmbd, npd)
    b = cbpdn.ConvBPDN(D, sh, lmbda, opt, dimK=0)
    X = b.solve()
    X = np.sum(abs(X), axis=b.cri.axisM).squeeze().flatten()
    # print(X.shape)
    return X

def wrapper(path, D):
    data = pd.read_csv(path)
    data = data.values
    # print(data[0])
    image=[]
    for i in range(data.shape[0]):
        i=data[i][0:-1].astype("float32")/255
        i=i.reshape(20,20)
        image.append(i)

    S = np.dstack(image)
    
    results = []
    for im in image:
        results.append(parse_repre(im, D))
    results = np.vstack(results)
    print(results.shape)

    labels = data[:,-1].astype("int64")
    print(labels.shape)

    results = np.column_stack((results, labels))
    print(results.shape)

    return results

In [87]:
for p in DATA_PATH:
    dic = generate_dict(TRAIN_DIR_PREFIX+p+".csv")
    with open("CCBRPDN_Dict_"+p+".obj", 'wb') as f:
        pickle.dump(dic, f)

ConvBPDNDictLearn solve time: 242.61s
ConvBPDNDictLearn solve time: 546.37s
ConvBPDNDictLearn solve time: 1037.86s
(2223, 400)
(2223,)
(2223, 401)
(5370, 400)
(5370,)
(5370, 401)
(10540, 400)
(10540,)
(10540, 401)
(556, 400)
(556,)
(556, 401)
(1343, 400)
(1343,)
(1343, 401)
(2635, 400)
(2635,)
(2635, 401)


In [13]:
for p in DATA_PATH:
    with open("CCBRPDN_Dict_"+p+".obj", 'rb') as f:
        D = pickle.load(f)
    train_sp = wrapper(TRAIN_DIR_PREFIX+p+".csv", D)
    np.savetxt("train_data_"+p+"_sparse.csv", train_sp, delimiter=",")

for p in DATA_PATH:
    with open("CCBRPDN_Dict_"+p+".obj", 'rb') as f:
        D = pickle.load(f)
    train_sp = wrapper(TEST_DIR_PREFIX+p+".csv", D)
    np.savetxt("test_data_"+p+"_sparse.csv", train_sp, delimiter=",")

(2223, 400)
(2223,)
(2223, 401)
(5370, 400)
(5370,)
(5370, 401)
(10540, 400)
(10540,)
(10540, 401)
(556, 400)
(556,)
(556, 401)
(1343, 400)
(1343,)
(1343, 401)
(2635, 400)
(2635,)
(2635, 401)
